# Biology 351 Lab: Week 7

## Processing sequencing reads and mapping to a reference genome

#### Introduction to this document:

You are reading a Jupyter notebook. It is similar to an R notebook in that we can intermix code and text.  But, since we need to run our code on the remote `spydur` computer, we will have to copy and paste it from this document to a Terminal in another tab of your browser. The `%%bash` at the top of each code cell indicates the type of code we are writing ("bash" is the programming language that allows us to interact with terminal). __Do not copy and paste the `%%bash` line of the code!__

Pro tip: triple-click on a line to highlight the entire line to copy.

## Logging on to spydur:

First let's log into `spydur`. Update the code below and use it to log in. If your key worked last week, you should not need to enter a password, but if you are prompted, enter your netid password



In [ ]:
%%bash
ssh <netid>@spydur

## Setting up folders
Now let's make new folders to work in for this week's lab. Depending on how things worked out last week, you may or may  not have a folder in the `/shared` directory on Spydur. Remember that the command to navigate is `cd` for change directory, and `mkdir` makes new directories. To move up a folder use `cd ..`  Use the code box below to navigate to `/home/netid/shared`, check to see if you have a folder. Make a folder with your first name if you don't. 

Everyone should make a subfolder below their first name called Lab_7. 

In the end you should have a folder at `/home/netid/shared/firstname/Lab7` (replacing your netid and first name).
    


In [ ]:
%%bash
#type code here as needed


## Variables in Unix

Just like we can save different object as variable names in R (think back to how we had separate data tables named "mydata" adn "mydata.melt", we can also save text, numbers, and objects to variable in UNIX. The formula is `variable_name=variable_value`. We can then refer back to those variables with a "${}" (we don't actually always need the {} but sometimes we do, so just to keep it consistent we will use them in this class). Try out the code below:

In [ ]:
#type your name immediately after the = with no spaces
name=
echo ${name} "loves learning bioinformatics!"

## Make a variable to save our working directory

Now we'll save this Lab7 folder as our "working directory". The working directory is where you have files and run code for the current analysis. We will save the directory as a variable name.  That way, we can just type \\${WD} instead of the full directory for everything we want to do. Update the code below with with the path to your Lab7 folder and run it. It should print back the location of ${WD}. Anytime you want to use a path to your current working directory, you can type \\${WD} insteads of `/home/netid/shared/firstname/Lab7`. Nice timesaver, huh?


In [2]:
%%bash
WD=/home/netid/shared/firstname/Lab7
WD=

echo ${WD}
cd ${WD}
pwd


/home/perickso/shared/DrE/Lab7


## Set up an interactive job
My lab (the Erickson lab) "owns" one computer that is part of the network of computesr that makes up the spydur cluster. We will be running our jobs on that computer.  To ensure that everyone's jobs run with equal computing resources, we will tell the computer how many cores of computing power and how much memory we need. Copy the code below and run it in terminal so that all your future commands run on my branch of `spydur`. This will give us each 20GB of memory on 3 computing cores to run our current operations. 


In [ ]:
%%bash

srun --pty -t 3:00:00 --mem=20G --partition erickson --ntasks-per-node=3 bash

## Finding the right files
We left off last week after looking at FastQ and FastA files.

### Question: What is the difference between the two file types?  What information is stored in each?

#### Answer here:

This week, we are going to take some of our sequencing reads and learn how to process them and map them to a reference genome.

Use the appropriate Unix command to look at the file in `shared` named `samples.txt`. You should be able to find the information you need to get the sample name. 

### Question: What are the names of the samples that you will be comparing? The name should be "ZP_" followed by a number

#### Answer here:

### Question: How many flies were squished up and pooled together for your samples?

#### Answer here:

### (Challenge!) Question: Will the number of flies affect how much sequencing data you have? Why or why not?

#### Answer here:


## Looking at the real fastq files

Our fastq files have a lot of data in them. The raw data are stored in a zipped, compressed form, indicated by a .gz at the end of the file.

### Question: Change directories to /shared/fastq. Use `ls -lh`  to see the sizes of all the files in bytes. G=gigabytes, M=megabytes, K=kilobytes. How big are the compressed raw data for your samples?

#### Answer here

One of these files might be bigger than your monthly cell phone data usage (or at least mine, I am old and don't use my phone very much!), which means they can take a long time to process. I ran a program that randomly downsampled the fastq files so that only 1% of sequencing the reads were retained. It was run equally on both the read_1 and read_2 files so that the same pairs of reads were retained in each file. 

### Question: how many lines are in the "downsampled" files for your samples? How many sequencing reads does that represent (remember from last week how reads are represented in a fastq file!)?

#### Answer here:

Using these downsampled files, we will be able to run examples of some of the key bioinformatics steps, without having to wait for a long time for each program to run.

Decide now with your partner who is going to process which file.

### Question: which sample will you be working with going forward?

#### Answer here:

#### Creating links to our mini-files
To save space on the server, we are not going to copy all the files to our individual working directories. Instead, we will use the `ln` function in unix to "link" them to our folders. These are like hyperlinks, and they represent shortcuts to the big files in your working folder so that you can pretend they are there without having to acctually copy the information in the file. Use the code below to set up links. That way, you can refer to the file as though it is in your working folder without having to type the full path to the file

In [ ]:
%%bash
#change directories to your Lab7 working folder using the shortcut you made above:

#link to read_1 file:

ln 

#link to read_2 file:

ln 


#now look at the contents of your Lab7 folder. Did the links work? 
#If they did, you should see two downsampled fastq files in your folder


#### Merging reads

Recall that we are using 150 bp, paired-end reads.

### Question: Describe in your own words what "paired end reads" means

#### Answer here:

### Question: Why might paired end reads occasionally overlap? What would have to be true for this to happen?

#### Answer here

Because some reads might overlap, the sequence in the overlapping part represents duplicated data that is not a true "replicate" of the sequencing information. We need to get rid of this duplication by merging the reads together. Essentially, the computer finds the overlap and comibines the two shorter overlapping reads into one long read.

The software that we are going to use to merge our reads is called `bbmap`. The specific program we are using is called `merge.sh`. The program takes two input files, the `read_1` and `read_2` fastq files. It finds occasions where the reads overlap, and merges them into a single read.

It has two input files that it will use:
 - in1: The read_1 reads (in fastq format)
 - in2: The read_2 reads (in fastq format)
 
It also has three output files that it will generate:
 - out: The merged reads
 - outu: The unmerged read 1 reads (ie the read 1 reads that didn't overlap with their paired end) 
 - outu2: The unmerged read 2 reads (ie the read 2 reads that didn't overlap with their paired end)
 
Set up the code below to run `bbmerge.sh` on one of your two downsampled files. Just like careful pipetting is critical to the success of a wet lab, proper file naming is going to be critical in this lab! We will assign each file name as a variable. The final line of code will execute the program with the proper files. Your output files should be saved in `WD` and named as follows. 
 
How to name your files:
 - out: \\${WD}/samplename.downsampled.merged.fq
 - outu: \\${WD}/samplename_1.downsampled.unmerged.fq
 - outu2: \\${WD}/samplename_2.downsampled.unmerged.fq
 
To simplify the line of code that actually runs the program we will save each of these names as a variable, then run the program just referencing the variables. When you enter the code below, there cannot be any spaces between the variable name, the =, and the file name!

In [9]:
%%bash

#set up the names of all your files here. don't forget you can use "${WD} as a shortcut!"

in1=
in2= 
out=
outu1=
outu2=

#copy and paste all 5 lines above into terminal

#then run this code to do the merging. 
#note that we are using the variable names that we set up above to tell the computer which files

bbmerge.sh in1=$in1 in2=$in2 out=$out outu1=$outu1 outu2=$outu2 strict=t





bash: line 3: bbmerge.sh: command not found
bash: line 5:  : command not found
bash: line 6:  : command not found
bash: line 7: -strict: command not found
bash: line 13: bbmerge.sh: command not found


CalledProcessError: Command 'b'#first edit for your first file\nbbmerge.sh \\\nin1=/home/perickso/private/bio351/ZP_1/ZP_1_1.downsampled.fq \\\nin2=/home/perickso/private/bio351/ZP_1/ZP_1_2.downsampled.fq \\ \nout=/home/perickso/private/bio351/ZP_1/ZP_1.downsampled.merged.fq \\ \noutu1=/home/perickso/private/bio351/ZP_1/ZP_1_1.downsampled.unmerged.fq \\ \noutu2=/home/perickso/private/bio351/ZP_1/ZP_1_2.downsampled.unmerged.fq \\\n-strict\n\n#now edit for your second file\n\nbbmerge.sh \\\n\tin1=$read1 \\\n    in2=$read2 \\\n\tout=$WORKING_FOLDER/merged_reads/${i}/${i}.merged.reads.strict.fq \\\n\toutu1=$WORKING_FOLDER/unmerged_reads/${i}/${i}.unmerged.reads.1.fq \\\n\toutu2=$WORKING_FOLDER/unmerged_reads/${i}/${i}.unmerged.reads.2.fq \\\n\t-strict\n'' returned non-zero exit status 127.

Take a look at the output that prints from bbmerge.sh. It reports a bunch of statistics about the merging process. Answer the questions below:

### Question: What % of your reads were merged together?

#### Answer here: 


### Question: The "insert size" tells you the average length of a molecule that was bound to the DNA sequencing machine. When I constructed these libraries, I aimed for the size of the molecules to be 500 bp. Did I achieve my goal? Why or why not?

#### Answer here: 

### Question: Sanity check: count the number of lines in your merged and unmerged fastq files (all 3 of them). Do the numbers make sense?

#### Answer here

Now it's time to map or align your fastq files to the reference genome. The program we will use is called `bwa`, which stands for Burrows-Wheeler aligner; named after the two people who invented the algorithm that it uses. Each person will run BWA twice: once on their merged reads, and once on their unmerged (still paired-end reads). 


### Question: how do you think the mapping algorithm works differently when it is using single reads (the merged reads) vs paired reads (the read_1 and read_2 files)?

#### Answer here



## Mapping the reads

 The overall syntax of how we run the program is

`bwa mem -M -t 3 reference_genome fastq_file (fastq_file_2) > output_file.sam`

 - `mem` is the specific algorithm the computer uses to map
 - `-M` is an option that makes the output files compatible with a program we will use in the future
 - `-t 3` tells the computer to use 3 computing cores
 - `reference_genome` is the file that contains our reference genome file (we looked at it last week)
 - `fastq_file` is the reads that you want to map
 - `fastq_file_2` is the optoinal paired end data for what you want to map
 - `output_file` is the name of the file we will save the mapping data to. It has to end in ".sam". Ideally it will have the same name as the input file but end in .sam instead of .fastq
 
 
Below you will need to set up the code to run the merged and paired end files through bwa. The second step is going to take 10-15 minutes to run most likely, so you can move on and start working on the next section once your code is running, then come back to answer the questions below about the output of bwa.

In [ ]:
%%bash

#set up the code below to map the merged reads

#path to the file with the reference genome (file we looked at last week in /shared)
ref=

# input merged read file
merged_reads=

#file you want to save to. needs to be descriptive and have .sam at the end of the name
merged_sam= 

#below is the code that will actually run the mapping
bwa mem -M -t 3 $ref $merged_reads > $merged_sam

#now set up the code below to map the unmerged reads

#location and name of  your unmerged read_1 fastq file (can copy from above)
read_1=

#location and name of your unmerged read_2 fastq file (can copy from above)
read_2=

#file that you want to save to. needs to have a descriptive name and end in .sam
unmerged_output=

bwa mem -M -t 3 $ref $read_1 $read_2 > $unmerged_output


Look at the output that printed to your screen after running each job:

### Question: how long did it take to run the two jobs?

#### Answer here

### Question: these reads where downsampled to 1% of their original read depth; how long would it have taken to run the whole file this way?

#### Answer here


### Question: look at your output .sam files (hint: use `less -S` to look at them in an orderly way that uses tabs to organize the columns). What key information does the sam file appear to contain? 

#### Answer here

    
### Question: read about the sam file format here: https://en.wikipedia.org/wiki/SAM_(file_format). What column would be important if you wanted to identify reads that were a poor match to the reference genome?

#### Answer here



## Converting to a bam file
Now, we will convert our sam files into a binary format file called a bam file. The information remains the same, but it is compressed and easier for the computer to work with. However, once it is in binary format, it is no longer human-readable. 

The command for to convert files to bam is called `samtool`s, and we will use a tool called  `view`. The option `-b` tells the computer to convert to bam. 

This is also the step where we can remove poorly mapped reads by setting a quality filter. The `-q 40` option below means that we will remove anything with a mapping quality lower than 40. This is the first of many important filtering steps that will happen!  You will need to repeat this step for both your merged and unmerged sam files. 


In [ ]:
%%bash
#the output named of your merged bam file. Same as the .sam name, but replace .sam with .bam
merged_bam_name=

samtools view -b -q 40 --threads 3  $merged_output > $merged_bam_name

unmerged_bam_name=${WD}/ZP_1.downsampled.unmerged.bam
samtools view -b -q 40 --threads 3  $unmerged_output > $unmerged_bam_name


### Question: Take a look at one of the bam files you just made. What does it look like inside? 

#### Answer here:

Now that we have the files in .bam format, they are easier for the computer to work with. Next we need to sort them. Sorting puts all the mapped reads in order along the reference genome. Sorting is required for many downsteram steps. The tool we will use is called `sort` and the program syntax is simple: 
`samtools sort input > output`


In [ ]:
%%bash
#generate a name to save your sorted file as
merged_sorted_filename=

#now write the command using the syntax above. 
#your input file is the same as the merged bam name you created above! 


#generate a name to save your other sorted file as. 
unmerged_sorted_filename=

#now write the command to make your other sorted file.


### Combine files
Now that our bam files are created and sorted, we can combine them into a final bam file containing all of our mapped reads. The syntax for this command is different; instead of using a > to tell the program where to save the file, the output file is the first argument. 



In [ ]:
%%bash
#add the name of your final file below
final_combined_bam=

#then write a command. the syntax is `samtools merge output_file file_1 file_2 


Ta da! We are done for today. We went from raw data to a sorted, combined file of mapped reads. Next week, we will use this file to identify the locations of SNPs in the genome. 

When you are done, hit file> print preview. Then print the resulting window to a PDF and upload to Blackboard. 